In [1]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

In [2]:
pd.set_option('display.max_columns', 100) # Para mostrar todas las columnas
# pd.set_option('display.max_rows', 100) # Para mostrar todas las filas

In [3]:
data_raw = pd.read_csv("Data/IMDb movies.csv", low_memory=False)

In [4]:
display(data_raw.head(1))
display(data_raw.shape)
display(data_raw.dtypes)
display(data_raw.columns)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0


(85855, 22)

imdb_title_id             object
title                     object
original_title            object
year                      object
date_published            object
genre                     object
duration                   int64
country                   object
language                  object
director                  object
writer                    object
production_company        object
actors                    object
description               object
avg_vote                 float64
votes                      int64
budget                    object
usa_gross_income          object
worlwide_gross_income     object
metascore                float64
reviews_from_users       float64
reviews_from_critics     float64
dtype: object

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics'],
      dtype='object')

In [5]:
data_raw.drop(['year', 'date_published','duration', 'director', 'writer',
              'production_company', 'actors','avg_vote', 'votes', 'budget', 'usa_gross_income',
              'worlwide_gross_income', 'metascore', 'imdb_title_id', 'original_title',
              'reviews_from_users', 'reviews_from_critics'], axis=1,inplace=True)

In [6]:
data_raw.head(1)

,title,genre,country,language,description
0,Miss Jerry,Romance,USA,None,The adventures of a female reporter in the 1890s.


In [7]:
countries = data_raw['country'].value_counts()

In [8]:
top_10_countries = countries[:10]
display(top_10_countries)
other_countries = countries[:20:-1]
#display(other_countries) 
#para dropear en caso de decidir no quedarnos con estos datos

USA          28511
India         6065
UK            4111
Japan         3077
France        3055
Italy         2444
Canada        1802
Germany       1396
Turkey        1351
Hong Kong     1239
Name: country, dtype: int64

In [9]:
languages = data_raw['language'].value_counts()
top_10_languages = languages[:10]

In [10]:
missing_values_check = data_raw.isnull().sum()
print(missing_values_check)

title             0
genre             0
country          64
language        833
description    2115
dtype: int64


In [11]:
data_raw.dropna(inplace=True)

In [12]:
missing_values_check = data_raw.isnull().sum()
print(missing_values_check)

title          0
genre          0
country        0
language       0
description    0
dtype: int64


In [13]:
mask_usa_uk_india = (data_raw['country']=='USA') | (data_raw['country']=='India') | (data_raw['country']=='UK')
data_usa = data_raw.loc[mask_usa_uk_india, :]
data_usa.head()

,title,genre,country,language,description
0,Miss Jerry,Romance,USA,None,The adventures of a female reporter in the 1890s.
3,Cleopatra,"Drama, History",USA,English,The fabled queen of Egypt's affair with Roman ...
5,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",USA,English,"An account of the life of Jesus Christ, based ..."
17,Amore di madre,Drama,USA,English,John Howard Payne at his most miserable point ...
19,Traffic in Souls,"Crime, Drama",USA,English,"A woman, with the aid of her police officer sw..."


In [14]:
data_usa.shape

(38078, 5)

In [15]:
data_usa['language'].value_counts()

English                                              28326
Hindi                                                 2065
Malayalam                                              818
Tamil                                                  812
English, Spanish                                       734
                                                     ...  
English, Latin, Spanish, French                          1
Marathi, Gujarati, English, Hindi                        1
English, Cantonese, Korean                               1
English, French, German, Greek, Ancient (to 1453)        1
English, Japanese, French, Arabic                        1
Name: language, Length: 947, dtype: int64

In [ ]:
#Detectar idioma y seleccionar solo lo que esté en inglés

# data_usa['descrip_lang'] = data_usa['description'].apply(lambda x: detect(str(str(x).split())[:5]))
# display(data_usa['descrip_lang'].value_counts())
# data_usa_en = data_usa.loc[data_usa['descrip_lang']=='en', :]
# data_usa_en.shape

In [16]:
#sacar si se corre la celda anterior

data_usa_en = data_usa

In [ ]:
# Si uso las funciones del notebook de Mark
#def complete_clean(sentence):
#     clean_html = cleanHtml(sentence)
#     clean_punc = cleanPunc(clean_html)
#     clean_alpha = keepAlpha(clean_punc)
#     return clean_alpha

In [17]:
import unidecode
import re

def clean_text(t):
    t_lower_no_accents=unidecode.unidecode(t.lower()); # sacamos acentos y llevamos a minuscula
    t_lower_no_accents_no_punkt=re.sub(r'([^\s\w]|_)+','',t_lower_no_accents); # quitamos signos de puntuacion usando una regex que reemplaza todo lo q no sean espacios o palabras por un string vacio
    return t_lower_no_accents_no_punkt

In [18]:
data_usa_en['description_clean'] = data_usa_en['description'].apply(clean_text)

/Users/Nakin/opt/anaconda3/envs/dhdsblend/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
data_usa_en.head()

,title,genre,country,language,description,description_clean
0,Miss Jerry,Romance,USA,None,The adventures of a female reporter in the 1890s.,the adventures of a female reporter in the 1890s
3,Cleopatra,"Drama, History",USA,English,The fabled queen of Egypt's affair with Roman ...,the fabled queen of egypts affair with roman g...
5,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",USA,English,"An account of the life of Jesus Christ, based ...",an account of the life of jesus christ based o...
17,Amore di madre,Drama,USA,English,John Howard Payne at his most miserable point ...,john howard payne at his most miserable point ...
19,Traffic in Souls,"Crime, Drama",USA,English,"A woman, with the aid of her police officer sw...",a woman with the aid of her police officer swe...


In [20]:
data_usa_en['genre_clean'] = data_usa_en['genre'].apply(clean_text)

/Users/Nakin/opt/anaconda3/envs/dhdsblend/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
data_usa_en.head(5)

,title,genre,country,language,description,description_clean,genre_clean
0,Miss Jerry,Romance,USA,None,The adventures of a female reporter in the 1890s.,the adventures of a female reporter in the 1890s,romance
3,Cleopatra,"Drama, History",USA,English,The fabled queen of Egypt's affair with Roman ...,the fabled queen of egypts affair with roman g...,drama history
5,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",USA,English,"An account of the life of Jesus Christ, based ...",an account of the life of jesus christ based o...,biography drama
17,Amore di madre,Drama,USA,English,John Howard Payne at his most miserable point ...,john howard payne at his most miserable point ...,drama
19,Traffic in Souls,"Crime, Drama",USA,English,"A woman, with the aid of her police officer sw...",a woman with the aid of her police officer swe...,crime drama


In [22]:
genres = pd.unique(data_usa_en['genre_clean'].str.split(expand=True).stack())
genres

array(['romance', 'drama', 'history', 'biography', 'crime', 'horror',
       'western', 'fantasy', 'comedy', 'family', 'adventure', 'action',
       'war', 'scifi', 'mystery', 'thriller', 'sport', 'musical', 'music',
       'filmnoir', 'animation', 'documentary', 'news', 'realitytv'],
      dtype=object)

In [23]:
data_usa_en['genre_list'] = data_usa_en['genre_clean'].str.split(" ")

/Users/Nakin/opt/anaconda3/envs/dhdsblend/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
data_usa_en.head(5)

,title,genre,country,language,description,description_clean,genre_clean,genre_list
0,Miss Jerry,Romance,USA,None,The adventures of a female reporter in the 1890s.,the adventures of a female reporter in the 1890s,romance,[romance]
3,Cleopatra,"Drama, History",USA,English,The fabled queen of Egypt's affair with Roman ...,the fabled queen of egypts affair with roman g...,drama history,"[drama, history]"
5,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",USA,English,"An account of the life of Jesus Christ, based ...",an account of the life of jesus christ based o...,biography drama,"[biography, drama]"
17,Amore di madre,Drama,USA,English,John Howard Payne at his most miserable point ...,john howard payne at his most miserable point ...,drama,[drama]
19,Traffic in Souls,"Crime, Drama",USA,English,"A woman, with the aid of her police officer sw...",a woman with the aid of her police officer swe...,crime drama,"[crime, drama]"


In [25]:
data_usa_en.shape

(38078, 8)

In [26]:
data_usa_en.reset_index(inplace=True, drop=True)

In [27]:
data_usa_en.head()

,title,genre,country,language,description,description_clean,genre_clean,genre_list
0,Miss Jerry,Romance,USA,None,The adventures of a female reporter in the 1890s.,the adventures of a female reporter in the 1890s,romance,[romance]
1,Cleopatra,"Drama, History",USA,English,The fabled queen of Egypt's affair with Roman ...,the fabled queen of egypts affair with roman g...,drama history,"[drama, history]"
2,"From the Manger to the Cross; or, Jesus of Naz...","Biography, Drama",USA,English,"An account of the life of Jesus Christ, based ...",an account of the life of jesus christ based o...,biography drama,"[biography, drama]"
3,Amore di madre,Drama,USA,English,John Howard Payne at his most miserable point ...,john howard payne at his most miserable point ...,drama,[drama]
4,Traffic in Souls,"Crime, Drama",USA,English,"A woman, with the aid of her police officer sw...",a woman with the aid of her police officer swe...,crime drama,"[crime, drama]"


In [28]:
from sklearn.preprocessing import MultiLabelBinarizer

In [29]:
binarizer = MultiLabelBinarizer(classes=genres)
binarizer.fit(data_usa_en['genre_list'])
genre_encoded = binarizer.transform(data_usa_en['genre_list'])


In [30]:
genre_enc = pd.DataFrame(genre_encoded, columns = binarizer.classes_, index = data_usa_en.index)

In [31]:
genre_enc.head(10)

,romance,drama,history,biography,crime,horror,western,fantasy,comedy,family,adventure,action,war,scifi,mystery,thriller,sport,musical,music,filmnoir,animation,documentary,news,realitytv
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
#count cantidad de movies por genero
counts = []
for genre in genre_enc.columns:
    counts.append((genre, genre_enc[genre].sum()))
df_stats = pd.DataFrame(counts, columns=['genre', 'number of movies'])
df_stats

,genre,number of movies
0,romance,7068
1,drama,19430
2,history,632
3,biography,929
4,crime,5615
5,horror,5434
6,western,1131
7,fantasy,1454
8,comedy,13020
9,family,1690


In [33]:
columns_to_drop = ['documentary', 'realitytv', 'news']
indexes_to_drop = []

for genre in columns_to_drop:
    genre = data_usa_en['genre_clean'].apply(lambda x: genre in str(x).lower());
    movies_index = list(genre[genre].index)
    for index in movies_index:
        indexes_to_drop.append(index)

In [34]:
indexes_to_drop

[12274, 35072, 31096]

In [35]:
display(data_usa_en.shape)
display(genre_enc.shape)

(38078, 8)

(38078, 24)

In [36]:
data_pre = pd.concat([data_usa_en,genre_enc], axis=1)
data_pre.sample(3)

,title,genre,country,language,description,description_clean,genre_clean,genre_list,romance,drama,history,biography,crime,horror,western,fantasy,comedy,family,adventure,action,war,scifi,mystery,thriller,sport,musical,music,filmnoir,animation,documentary,news,realitytv
19410,Kissi Se Na Kehna,"Comedy, Romance",India,Hindi,"Kailashnath is a widower, and father of an onl...",kailashnath is a widower and father of an only...,comedy romance,"[comedy, romance]",1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33208,The Incident,"Drama, Thriller",UK,English,"A successful young couple's sheltered, affluen...",a successful young couples sheltered affluent ...,drama thriller,"[drama, thriller]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
23473,Reign Over Me,Drama,USA,English,A man who lost his family in the September 11 ...,a man who lost his family in the september 11 ...,drama,[drama],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
data_pre.shape

(38078, 32)

In [38]:
#Drop de filas donde hay movies de las 4 ultimas categorias
data_pre.drop(index=indexes_to_drop, inplace=True)

In [39]:
counts_2 = []
for genre in genre_enc.columns:
    counts_2.append((genre, data_pre[genre].sum()))
df_stats_2 = pd.DataFrame(counts_2, columns=['genre', 'number of movies'])
df_stats_2

,genre,number of movies
0,romance,7068
1,drama,19429
2,history,632
3,biography,928
4,crime,5615
5,horror,5433
6,western,1131
7,fantasy,1454
8,comedy,13020
9,family,1690


In [40]:
#Drop las columnas de adult, documentary, reality_tv y news
data_pre.drop(columns=columns_to_drop, axis=1, inplace=True)

In [41]:
data_pre.sample(3)

,title,genre,country,language,description,description_clean,genre_clean,genre_list,romance,drama,history,biography,crime,horror,western,fantasy,comedy,family,adventure,action,war,scifi,mystery,thriller,sport,musical,music,filmnoir,animation
30588,Kumki,Drama,India,Tamil,The love between a mahout and an elephant.,the love between a mahout and an elephant,drama,[drama],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37820,Premature,"Drama, Music, Romance",USA,English,Seventeen year-old Ayanna meets handsome and m...,seventeen yearold ayanna meets handsome and my...,drama music romance,"[drama, music, romance]",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
36490,Pandigai,"Action, Thriller",India,Tamil,A low-level hotel worker gets involved in a st...,a lowlevel hotel worker gets involved in a str...,action thriller,"[action, thriller]",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [ ]:
#check si las filas suman cero 

In [42]:
mask = genre_enc.sum(axis=1) == 0

In [43]:
len(mask)

38078

In [44]:
any(mask)

False

In [45]:
data_pre.columns

Index(['title', 'genre', 'country', 'language', 'description',
       'description_clean', 'genre_clean', 'genre_list', 'romance', 'drama',
       'history', 'biography', 'crime', 'horror', 'western', 'fantasy',
       'comedy', 'family', 'adventure', 'action', 'war', 'scifi', 'mystery',
       'thriller', 'sport', 'musical', 'music', 'filmnoir', 'animation'],
      dtype='object')

In [46]:
data_final = data_pre.drop(['genre', 'country', 'language', 'description', 'genre_list'], axis=1)
data_final.sample(3)

,title,description_clean,genre_clean,romance,drama,history,biography,crime,horror,western,fantasy,comedy,family,adventure,action,war,scifi,mystery,thriller,sport,musical,music,filmnoir,animation
1380,Il mistero del ranch,kentucky loses his horse in a rigged horse rac...,action crime music,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
29482,The Deadly Game - Gioco pericoloso,a thief is caught up in a deadly game of catan...,action crime drama,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
8339,Il piacere della disonestà,a poor but proud french teacher gets fired aft...,comedy drama,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
data_final.to_csv(path_or_buf='Data/movies_usa_en.csv', sep=',',
                   header=True, encoding='utf8', index=False)